# Initial Modelling with XGBoost

In [18]:
import pandas as pd
import glob
import sweetviz as sv
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, plot_roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score,accuracy_score
import numpy as np

In [19]:
pd.set_option('display.max_columns', None)

In [58]:
fixtures_total

,ascore,hgoals,winner,roundname,round,localtime,unixtime,hteamid,hteam,hbehinds,updated,hscore,is_grand_final,date,timestr,venue,winnerteamid,tz,year,agoals,complete,id,ateamid,is_final,abehinds,ateam
0,83,12,Fremantle,Round 1,1,2022-03-20 15:40:00,1.647753e+09,1,Adelaide,10,2022-03-20 18:50:03,82,0,2022-03-20 16:10:00,Full Time,Adelaide Oval,6.0,+11:00,2022,11,100,8669,6,0,17,Fremantle
1,69,11,Brisbane Lions,Round 1,1,2022-03-19 19:10:00,1.647681e+09,2,Brisbane Lions,14,2022-03-19 22:56:03,80,0,2022-03-19 20:10:00,Full Time,Gabba,2.0,+11:00,2022,10,100,8667,13,0,9,Port Adelaide
2,76,14,Carlton,Round 1,1,2022-03-17 19:25:00,1.647506e+09,3,Carlton,17,2022-03-17 22:02:04,101,0,2022-03-17 19:25:00,Full Time,M.C.G.,3.0,+11:00,2022,11,100,8663,14,0,10,Richmond
3,72,20,Geelong,Round 1,1,2022-03-19 14:10:00,1.647659e+09,7,Geelong,18,2022-03-19 16:56:02,138,0,2022-03-19 14:10:00,Full Time,M.C.G.,7.0,+11:00,2022,11,100,8665,5,0,6,Essendon
4,112,13,Sydney,Round 1,1,2022-03-19 17:10:00,1.647670e+09,9,Greater Western Sydney,14,2022-03-19 19:56:02,92,0,2022-03-19 17:10:00,Full Time,Stadium Australia,16.0,+11:00,2022,17,100,8666,16,0,10,Sydney
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,59,9,Collingwood,Semi-Final,25,2018-09-15 19:25:00,NaN,4,Collingwood,15,2018-09-15 21:52:02,69,0,2018-09-15 19:25:00,NaN,M.C.G.,4.0,+10:00,2018,9,100,2961,9,4,5,Greater Western Sydney
203,104,10,Melbourne,Semi-Final,25,2018-09-14 19:50:00,NaN,10,Hawthorn,11,2018-09-14 22:22:03,71,0,2018-09-14 19:50:00,NaN,M.C.G.,11.0,+10:00,2018,16,100,2960,11,4,8,Melbourne
204,97,8,Collingwood,Preliminary Final,26,2018-09-21 19:50:00,NaN,14,Richmond,10,2018-09-21 22:26:02,58,0,2018-09-21 19:50:00,NaN,M.C.G.,4.0,+10:00,2018,15,100,2998,4,5,7,Collingwood
205,55,18,West Coast,Preliminary Final,26,2018-09-22 13:20:00,NaN,17,West Coast,13,2018-09-22 18:00:03,121,0,2018-09-22 15:20:00,NaN,Perth Stadium,17.0,+10:00,2018,7,100,2999,11,5,13,Melbourne


In [59]:
fixtures_list = glob.glob('./Data/Fixtures/*.csv')
frame = []
for file in fixtures_list:
    frame.append(pd.read_csv(file))

fixtures_total = pd.concat(frame)

fixtures_total_cleaned = fixtures_total.dropna(subset=['winner'])
fixtures_total_cleaned = fixtures_total_cleaned[fixtures_total_cleaned.is_grand_final == 0]
fixtures_total_cleaned = fixtures_total_cleaned[fixtures_total_cleaned.is_final == 0]

fixtures_total_cleaned = fixtures_total_cleaned.astype({'winnerteamid': int})

fixtures_usable = fixtures_total_cleaned.drop(columns=['localtime','winner', 'is_grand_final', 'is_final', 'unixtime','id','hteam','ateam','timestr','roundname','updated','complete','ateam'])

fixtures_usable['date'] = pd.to_datetime(fixtures_usable['date'])

fixtures_usable['day_of_week'] = fixtures_usable['date'].dt.day_of_week
fixtures_usable['day_of_year'] = fixtures_usable['date'].dt.day_of_year
fixtures_usable['is_month_end'] = fixtures_usable['date'].dt.is_month_end
fixtures_usable['is_month_start'] = fixtures_usable['date'].dt.is_month_start
fixtures_usable['is_quarter_end'] = fixtures_usable['date'].dt.is_quarter_end
fixtures_usable['is_quarter_start'] = fixtures_usable['date'].dt.is_quarter_start
fixtures_usable['month'] = fixtures_usable['date'].dt.month
fixtures_usable['day'] = fixtures_usable['date'].dt.day
fixtures_usable['hour'] = fixtures_usable['date'].dt.hour
fixtures_usable['minute'] = fixtures_usable['date'].dt.minute

fixtures_usable.replace({False: 0, True: 1}, inplace=True)
fixtures_usable = fixtures_usable.sort_values(by=['date'], ascending=True)
fixtures_usable = fixtures_usable.drop(columns=['date'])

In [44]:
ladder_list = glob.glob('./Data/Ladder/*.csv')

table_list = []

ladder_df_col_values = list(pd.read_csv(ladder_list[0]).columns.values)
ladder_df_col_values.append('Year')
ladder_df_col_values.append('Round')

frame_count = 0
for file in ladder_list:
    frame_pd = pd.DataFrame(pd.read_csv(file))
    
    year_round = file.split(".")[1]
    year_round = year_round.replace('/Data/Ladder/Ladder_','')
    year_round = year_round.split('_')
    
    frame_pd['Year'] = year_round[0]
    frame_pd['Round'] = year_round[2]
    
    frame_pd = frame_pd.astype({"Year": int, "Round": int})
    
    if frame_count == 0:
        ladder_df_total = frame_pd.copy()
    else:    
        ladder_df_total = pd.concat([ladder_df_total, frame_pd])
    
    frame_count += 1

In [24]:
lineup_list = glob.glob('./Data/Lineups/*.csv')

frame = []
for file in lineup_list:
    frame.append(pd.read_csv(file))

lineup_total = pd.concat(frame)

In [25]:
player_stats_list = glob.glob('./Data/Player_Stats/*.csv')

frame = []
for file in player_stats_list:
    frame.append(pd.read_csv(file, low_memory=False))

player_stats_total = pd.concat(frame)

In [26]:
results_list = glob.glob('./Data/Results/*.csv')

frame = []
for file in results_list:
    frame.append(pd.read_csv(file, low_memory=False))

results_total = pd.concat(frame)

In [27]:
team_id_to_name = dict(zip(fixtures_total_cleaned.hteam,fixtures_total_cleaned.hteamid))
team_id_to_name_sorted = sorted(team_id_to_name.items(), key=lambda x:x[1], reverse=False)

In [45]:
fixtures_usable.tail()

,ascore,hgoals,round,hteamid,hbehinds,hscore,is_grand_final,venue,winnerteamid,tz,year,agoals,ateamid,is_final,abehinds,day_of_week,day_of_year,is_month_end,is_month_start,is_quarter_end,is_quarter_start,month,day,hour,minute
203,92,11,25,11,13,79,0,M.C.G.,2,+10:00,2022,14,2,4,8,4,252,0,0,0,0,9,9,19,50
202,59,11,25,4,13,79,0,M.C.G.,4,+10:00,2022,9,6,4,5,5,253,0,0,0,0,9,10,19,25
204,49,18,26,7,12,120,0,M.C.G.,7,+10:00,2022,7,2,5,7,4,259,0,0,0,0,9,16,19,50
205,94,14,26,16,11,95,0,S.C.G.,16,+10:00,2022,14,4,5,10,5,260,0,0,0,0,9,17,16,45
206,52,20,27,7,13,133,1,M.C.G.,7,+10:00,2022,8,16,6,4,5,267,0,0,0,0,9,24,14,30


In [62]:
ladder_df_total

,wins,percentage,id,for.,pts,rank,against,goals_for,draws,behinds_against,behinds_for,goals_against,losses,name,played,Year,Round
0,6,160.878244,1,806,24,1.0,501,121,0,51,80,75,0,Adelaide,6,2017,6
1,5,133.333333,9,652,20,2.0,489,95,0,75,82,69,1,Greater Western Sydney,6,2017,6
2,5,131.870229,7,691,20,3.0,524,105,0,80,61,74,1,Geelong,6,2017,6
3,5,112.015504,14,578,20,4.0,516,84,0,66,74,75,1,Richmond,6,2017,6
4,4,152.155172,13,706,16,5.0,464,104,0,62,82,67,2,Port Adelaide,6,2017,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,3,86.607143,9,776,12,14.0,896,111,0,110,110,131,7,Greater Western Sydney,10,2022,10
14,3,80.169671,1,756,12,15.0,943,109,0,109,102,139,7,Adelaide,10,2022,10
15,2,73.353293,5,735,8,16.0,1002,105,0,120,105,147,8,Essendon,10,2022,10
16,1,53.644860,12,574,4,17.0,1070,84,0,134,70,156,9,North Melbourne,10,2022,10


In [60]:
combined_data = pd.merge(fixtures_usable, ladder_df_total,  how='left', left_on=['hteamid','round','year'], right_on = ['id','Round','Year'])

In [61]:
combined_data.tail()

,ascore,hgoals,round,hteamid,hbehinds,hscore,venue,winnerteamid,tz,year,agoals,ateamid,abehinds,day_of_week,day_of_year,is_month_end,is_month_start,is_quarter_end,is_quarter_start,month,day,hour,minute,wins,percentage,id,for.,pts,rank,against,goals_for,draws,behinds_against,behinds_for,goals_against,losses,name,played,Year,Round
1915,141,11,23,5,9,75,M.C.G.,14,+10:00,2022,21,14,15,5,232,0,0,0,0,8,20,19,25,7,83.229516,5,1737,28,15.0,2087,249,0,245,243,307,15,Essendon,22,2022,23
1916,55,16,23,13,15,111,Adelaide Oval,13,+10:00,2022,7,1,13,5,232,0,0,0,0,8,20,19,30,10,110.256410,13,1806,40,11.0,1638,261,0,240,240,233,12,Port Adelaide,22,2022,23
1917,87,10,23,10,4,64,York Park,18,+10:00,2022,12,18,15,6,233,0,0,0,0,8,21,13,10,8,89.753893,10,1787,32,13.0,1991,262,0,245,215,291,14,Hawthorn,22,2022,23
1918,75,10,23,3,14,74,M.C.G.,4,+10:00,2022,11,4,9,6,233,0,0,0,0,8,21,15,20,12,108.343057,3,1857,48,9.0,1714,268,0,226,249,248,10,Carlton,22,2022,23
1919,88,11,23,15,8,74,Docklands,16,+10:00,2022,13,16,10,6,233,0,0,0,0,8,21,16,40,11,99.300292,15,1703,44,10.0,1715,247,0,233,221,247,11,St Kilda,22,2022,23


# Feature Engineering with fixtures

In [30]:
home_features = ['hgoals', 'hbehinds', 'hscore']
away_features = ['agoals', 'abehinds', 'ascore']
def create_exp_weighted_avgs(df, span):
    ema_features_home = fixtures_usable[['id','hteamid']].copy()
        # Create a copy of the df
    for feat_name in home_features:
        feature_ema = (df.groupby('hteamid')[feat_name]
                        .transform(lambda row: (row.ewm(span-span)
                                                .mean()
                                                .shift(1))))
        feat_name = 'ewm_' + str(span) + '_' + feat_name
        ema_features_home[feat_name] = feature_ema
    
    ema_features_away = fixtures_usable[['id','ateamid']].copy()
    for feat_name in away_features:
        feature_ema = (df.groupby('ateamid')[feat_name]
                        .transform(lambda row: (row.ewm(span-span)
                                                .mean()
                                                .shift(1))))
        feat_name = 'ewm_' + str(span) + '_' + feat_name
        ema_features_away[feat_name] = feature_ema
    
    return ema_features_home, ema_features_away

features_rolling_averages_home, features_rolling_averages_away = create_exp_weighted_avgs(fixtures_usable, span=10)

features_rolling_averages_home.drop(columns=['hteamid'], inplace=True)
features_rolling_averages_away.drop(columns=['ateamid'], inplace=True)

fixtures_usable_merged = pd.merge(fixtures_usable, features_rolling_averages_home, on="id", how='left')
fixtures_usable_merged = pd.merge(fixtures_usable, features_rolling_averages_away, on="id", how='left')

In [31]:
fixtures_usable_merged

,ascore,hgoals,round,hteamid,hbehinds,hscore,is_grand_final,venue,winnerteamid,tz,year,agoals,id,ateamid,is_final,abehinds,day_of_week,day_of_year,is_month_end,is_month_start,is_quarter_end,is_quarter_start,month,day,hour,minute,ewm_10_agoals,ewm_10_abehinds,ewm_10_ascore
0,117,11,1,1,16,82,0,Football Park,5,+11:00,2013,18,1826,5,0,9,4,81,0,0,0,0,3,22,20,40,NaN,NaN,NaN
1,80,16,1,6,12,108,0,Subiaco,6,+11:00,2013,11,1828,17,0,14,5,82,0,0,0,0,3,23,19,40,NaN,NaN,NaN
2,106,14,1,3,17,101,0,M.C.G.,14,+11:00,2013,14,1829,14,0,22,3,87,0,0,0,0,3,28,19,40,NaN,NaN,NaN
3,59,19,1,18,13,127,0,Docklands,18,+11:00,2013,7,1830,2,0,17,5,89,0,0,0,0,3,30,13,40,NaN,NaN,NaN
4,106,11,1,9,10,76,0,Stadium Australia,16,+11:00,2013,16,1831,16,0,10,5,89,0,0,0,0,3,30,16,40,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,92,11,25,11,13,79,0,M.C.G.,2,+10:00,2022,14,10910,2,4,8,4,252,0,0,0,0,9,9,19,50,12.0,9.0,81.0
2006,59,11,25,4,13,79,0,M.C.G.,4,+10:00,2022,9,10909,6,4,5,5,253,0,0,0,0,9,10,19,25,13.0,11.0,89.0
2007,49,18,26,7,12,120,0,M.C.G.,7,+10:00,2022,7,10911,2,5,7,4,259,0,0,0,0,9,16,19,50,14.0,8.0,92.0
2008,94,14,26,16,11,95,0,S.C.G.,16,+10:00,2022,14,10912,4,5,10,5,260,0,0,0,0,9,17,16,45,10.0,12.0,72.0


In [32]:
fixtures_usable_training = fixtures_usable_merged.drop(columns=['ascore','tz','id','hgoals','hbehinds','abehinds','hscore','agoals'])
fixtures_usable_training.dropna(inplace=True)
fixtures_usable_training['home_team_winner'] = fixtures_usable_training.apply(lambda x : 1 if x['winnerteamid'] == x['hteamid'] else 0, axis=1)
fixtures_usable_training = fixtures_usable_training.drop(columns=['winnerteamid'])
fixtures_usable_training = pd.get_dummies(fixtures_usable_training)

In [33]:
fixtures_usable_training

,round,hteamid,is_grand_final,year,ateamid,is_final,day_of_week,day_of_year,is_month_end,is_month_start,is_quarter_end,is_quarter_start,month,day,hour,minute,ewm_10_agoals,ewm_10_abehinds,ewm_10_ascore,home_team_winner,venue_Adelaide Arena at Jiangwan Stadium,venue_Adelaide Oval,venue_Bellerive Oval,venue_Carrara,venue_Cazaly's Stadium,venue_Docklands,venue_Eureka Stadium,venue_Football Park,venue_GMHBA Stadium,venue_Gabba,venue_Jiangwan Stadium,venue_Kardinia Park,venue_M.C.G.,venue_Manuka Oval,venue_Marrara Oval,venue_Mars Stadium,venue_Marvel Stadium,venue_Optus Stadium,venue_Perth Stadium,venue_Riverway Stadium,venue_S.C.G.,venue_Stadium Australia,venue_Subiaco,venue_Sydney Showground,venue_Traeger Park,venue_UNSW Canberra Oval,venue_University of Tasmania Stadium,venue_Wellington,venue_York Park
9,2,15,0,2013,14,0,4,95,0,0,0,0,4,5,19,50,14.0,22.0,106.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,3,6,0,2013,5,0,4,102,0,0,0,0,4,12,20,40,18.0,9.0,117.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
19,3,12,0,2013,16,0,5,103,0,0,0,0,4,13,13,45,16.0,10.0,106.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20,3,11,0,2013,17,0,5,103,0,0,0,0,4,13,14,10,11.0,14.0,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
21,3,9,0,2013,15,0,5,103,0,0,0,0,4,13,16,40,10.0,17.0,77.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,25,11,0,2022,2,4,4,252,0,0,0,0,9,9,19,50,12.0,9.0,81.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2006,25,4,0,2022,6,4,5,253,0,0,0,0,9,10,19,25,13.0,11.0,89.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2007,26,7,0,2022,2,5,4,259,0,0,0,0,9,16,19,50,14.0,8.0,92.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2008,26,16,0,2022,4,5,5,260,0,0,0,0,9,17,16,45,10.0,12.0,72.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


# Feature Engineering with Player Stats

In [11]:
print(player_stats_total.columns.values)

['venue_name' 'match_id' 'match_home_team' 'match_away_team' 'match_date'
 'match_local_time' 'match_attendance' 'match_round'
 'match_home_team_goals' 'match_home_team_behinds' 'match_home_team_score'
 'match_away_team_goals' 'match_away_team_behinds' 'match_away_team_score'
 'match_margin' 'match_winner' 'match_weather_temp_c' 'match_weather_type'
 'player_id' 'player_first_name' 'player_last_name' 'player_height_cm'
 'player_weight_kg' 'player_is_retired' 'player_team' 'guernsey_number'
 'kicks' 'marks' 'handballs' 'disposals' 'effective_disposals'
 'disposal_efficiency_percentage' 'goals' 'behinds' 'hitouts' 'tackles'
 'rebounds' 'inside_fifties' 'clearances' 'clangers' 'free_kicks_for'
 'free_kicks_against' 'brownlow_votes' 'contested_possessions'
 'uncontested_possessions' 'contested_marks' 'marks_inside_fifty'
 'one_percenters' 'bounces' 'goal_assists' 'time_on_ground_percentage'
 'afl_fantasy_score' 'supercoach_score' 'centre_clearances'
 'stoppage_clearances' 'score_involvemen

In [ ]:
fixtures_usable_training[fixtures_usable_training['year'] == 2022]

In [12]:
player_stats_total.head()

,venue_name,match_id,match_home_team,match_away_team,match_date,match_local_time,match_attendance,match_round,match_home_team_goals,match_home_team_behinds,match_home_team_score,match_away_team_goals,match_away_team_behinds,match_away_team_score,match_margin,match_winner,match_weather_temp_c,match_weather_type,player_id,player_first_name,player_last_name,player_height_cm,player_weight_kg,player_is_retired,player_team,guernsey_number,kicks,marks,handballs,disposals,effective_disposals,disposal_efficiency_percentage,goals,behinds,hitouts,tackles,rebounds,inside_fifties,clearances,clangers,free_kicks_for,free_kicks_against,brownlow_votes,contested_possessions,uncontested_possessions,contested_marks,marks_inside_fifty,one_percenters,bounces,goal_assists,time_on_ground_percentage,afl_fantasy_score,supercoach_score,centre_clearances,stoppage_clearances,score_involvements,metres_gained,turnovers,intercepts,tackles_inside_fifty,contest_def_losses,contest_def_one_on_ones,contest_off_one_on_ones,contest_off_wins,def_half_pressure_acts,effective_kicks,f50_ground_ball_gets,ground_ball_gets,hitouts_to_advantage,hitout_win_percentage,intercept_marks,marks_on_lead,pressure_acts,rating_points,ruck_contests,score_launches,shots_at_goal,spoils,subbed,player_position,date
0,MCG,15615,Richmond,Carlton,2020-03-19,19:40:00,0,1,16,9,105,12,9,81,24,Richmond,30,RAIN,11285,Kade,Simpson,183.0,73.0,False,Carlton,6,13,6,7,20,17.0,85,0,0,0,0,6,0,3,3,1,0,0,6,11,0,0,0,0,0,88,NaN,NaN,1,1,4.0,376.0,5.0,8.0,0.0,0.0,1.0,0.0,0.0,7.0,10.0,0.0,5.0,0.0,0.0,2.0,0.0,7.0,6.5,0.0,2.0,0.0,0.0,Not Subbed,INT,2020-03-19
1,MCG,15615,Richmond,Carlton,2020-03-19,19:40:00,0,1,16,9,105,12,9,81,24,Richmond,30,RAIN,11472,Marc,Murphy,180.0,81.0,False,Carlton,3,12,8,14,26,18.0,69,0,0,0,0,0,6,5,7,2,4,0,11,18,0,2,2,0,1,87,NaN,NaN,1,4,7.0,347.0,7.0,2.0,0.0,0.0,0.0,1.0,0.0,7.0,8.0,1.0,8.0,0.0,0.0,0.0,0.0,17.0,3.8,0.0,1.0,1.0,0.0,Not Subbed,RR,2020-03-19
2,MCG,15615,Richmond,Carlton,2020-03-19,19:40:00,0,1,16,9,105,12,9,81,24,Richmond,30,RAIN,11561,Shane,Edwards,182.0,78.0,False,Richmond,10,7,1,8,15,13.0,87,1,0,0,2,0,6,5,2,1,1,0,6,10,0,0,2,0,2,90,NaN,NaN,4,1,8.0,312.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,5.0,0.0,2.0,0.0,0.0,0.0,0.0,16.0,13.2,0.0,0.0,2.0,1.0,Not Subbed,CHF,2020-03-19
3,MCG,15615,Richmond,Carlton,2020-03-19,19:40:00,0,1,16,9,105,12,9,81,24,Richmond,30,RAIN,11582,Jack,Riewoldt,193.0,92.0,False,Richmond,8,5,5,1,6,4.0,67,3,1,0,2,0,1,0,6,0,3,0,3,3,3,5,2,0,1,97,NaN,NaN,0,0,5.0,133.0,5.0,0.0,1.0,0.0,0.0,5.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,7.0,5.9,2.0,0.0,4.0,2.0,Not Subbed,FPR,2020-03-19
4,MCG,15615,Richmond,Carlton,2020-03-19,19:40:00,0,1,16,9,105,12,9,81,24,Richmond,30,RAIN,11651,Matthew,Kreuzer,200.0,101.0,False,Carlton,8,0,1,1,1,1.0,100,0,0,7,1,0,0,0,0,0,0,0,0,1,0,0,2,0,0,21,NaN,NaN,0,0,1.0,-15.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,4.0,46.7,1.0,0.0,4.0,3.2,15.0,1.0,0.0,1.0,Not Subbed,RK,2020-03-19


In [15]:
player_stats_total[player_stats_total['player_last_name'] == 'Jaksch']

,venue_name,match_id,match_home_team,match_away_team,match_date,match_local_time,match_attendance,match_round,match_home_team_goals,match_home_team_behinds,match_home_team_score,match_away_team_goals,match_away_team_behinds,match_away_team_score,match_margin,match_winner,match_weather_temp_c,match_weather_type,player_id,player_first_name,player_last_name,player_height_cm,player_weight_kg,player_is_retired,player_team,guernsey_number,kicks,marks,handballs,disposals,effective_disposals,disposal_efficiency_percentage,goals,behinds,hitouts,tackles,rebounds,inside_fifties,clearances,clangers,free_kicks_for,free_kicks_against,brownlow_votes,contested_possessions,uncontested_possessions,contested_marks,marks_inside_fifty,one_percenters,bounces,goal_assists,time_on_ground_percentage,afl_fantasy_score,supercoach_score,centre_clearances,stoppage_clearances,score_involvements,metres_gained,turnovers,intercepts,tackles_inside_fifty,contest_def_losses,contest_def_one_on_ones,contest_off_one_on_ones,contest_off_wins,def_half_pressure_acts,effective_kicks,f50_ground_ball_gets,ground_ball_gets,hitouts_to_advantage,hitout_win_percentage,intercept_marks,marks_on_lead,pressure_acts,rating_points,ruck_contests,score_launches,shots_at_goal,spoils,subbed,player_position,date
8271,GIANTS Stadium,14354,Greater Western Sydney,Richmond,2013-08-25,15:20:00,12314,22,6,6,42,25,13,163,121,Richmond,23,MOSTLY_SUNNY,12234,Kristian,Jaksch,NaN,NaN,True,Greater Western Sydney,34,6,6,4,10,9.0,90,1,0,0,1,0,0,0,1,0,0,0,2,8,0,0,3,0,0,87,54.0,51.0,0,0,2.0,59.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,4.0,5.0,1.0,2.0,0.0,0.0,0.0,1.0,5.0,6.3,0.0,0.0,1.0,3.0,Not Subbed,FPL,2013-08-25
8622,Metricon Stadium,14362,Gold Coast,Greater Western Sydney,2013-09-01,13:10:00,13080,23,22,14,146,9,9,63,83,Gold Coast,25,OVERCAST,12234,Kristian,Jaksch,NaN,NaN,True,Greater Western Sydney,34,4,2,1,5,3.0,60,0,0,0,0,0,1,0,1,0,1,0,1,4,0,0,0,0,1,31,17.0,15.0,0,0,2.0,121.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,4.0,1.1,0.0,2.0,0.0,0.0,Subbed In,SUB,2013-09-01
39,MCG,14581,Carlton,Richmond,2015-04-02,19:20:00,83493,1,11,12,78,15,15,105,27,Richmond,17,OVERCAST,12234,Kristian,Jaksch,NaN,NaN,True,Carlton,18,5,5,0,5,3.0,60,1,0,0,1,1,3,0,4,0,4,0,1,4,1,1,6,0,0,88,28.0,30.0,0,0,2.0,199.0,2.0,0.0,0.0,2.0,5.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.3,0.0,0.0,1.0,6.0,Not Subbed,BPL,2015-04-02
1622,MCG,14617,Carlton,Collingwood,2015-05-01,19:50:00,71759,5,6,9,45,18,12,120,75,Collingwood,21,MOSTLY_SUNNY,12234,Kristian,Jaksch,NaN,NaN,True,Carlton,18,4,3,4,8,6.0,75,0,0,0,1,3,0,0,0,0,0,0,2,5,0,0,11,0,0,96,33.0,44.0,0,1,1.0,162.0,2.0,2.0,0.0,2.0,6.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,9.4,0.0,1.0,0.0,11.0,Not Subbed,BPR,2015-05-01
2322,Marvel Stadium,14633,Carlton,Brisbane Lions,2015-05-10,15:20:00,20273,6,11,9,75,12,12,84,9,Brisbane Lions,15,RAIN,12234,Kristian,Jaksch,NaN,NaN,True,Carlton,18,9,7,9,18,12.0,67,0,0,0,1,3,1,0,3,0,0,0,3,16,0,0,6,0,0,91,70.0,70.0,0,0,3.0,294.0,6.0,6.0,0.0,0.0,4.0,0.0,0.0,2.0,5.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,3.1,0.0,1.0,0.0,5.0,Not Subbed,FB,2015-05-10
2548,Marvel Stadium,14638,Carlton,Greater Western Sydney,2015-05-16,16:35:00,16676,7,9,3,57,19,21,135,78,Greater Western Sydney,17,OVERCAST,12234,Kristian,Jaksch,NaN,NaN,True,Carlton,18,4,2,3,7,5.0,71,0,0,0,2,3,0,0,0,0,0,0,0,5,0,0,3,0,0,87,32.0,38.0,0,0,0.0,136.0,1.0,1.0,0.0,0.0,2.0,2.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.3,0.0,0.0,0.0,3.0,Not Subbed,BPL,2015-05-16
7290,Gabba,14746,Brisbane Lions,Carlton,2015-08-15,19:20:00,17744,20,20,11,131,9,13,67,64,Brisbane Lions,23,MOSTLY_SUNNY,12234,Kristian,Jaksch,NaN,NaN,True,Carlton,18,9,6,3,12,9.0,75,0,2,0,2,3,3,0,1,1,0,0,4,8,1,1,3,0,0,92,62.0,70.0,0,0,3.0,288.0,3.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,7.0,0.0,2.0,0.0,0.0,0.0,2.0,6.0,5.2,0.0,1.0,2.0,3.0,Not Subbed,BPR,2015-08-15
7821,MCG,14758,Carlton,Melbourne,2015-08-23,15:20:00,33962,21,12,6,78,7,13,55,23,Carlton,15,OVERCAST,12234,Kristian,Jaksch,NaN,NaN,True,Carlton,18,8,3,4,12,8.0,67,0,0,0,1,2,0,0,3,1,1,0,4,8,1,0,7

In [63]:
player_stats_total.groupby(['match_id', 'match_home_team'])

In [14]:
lineup_total[lineup_total['player.playerName.surname'] == 'Jaksch']

,providerId,utcStartTime,status,compSeason.shortName,round.name,round.roundNumber,venue.name,teamAbbr,teamName,teamNickname,teamId,position,player.playerId,player.captain,player.playerJumperNumber,player.playerName.givenName,player.playerName.surname,teamStatus,teamType,lateChanges
0,CD_M20150140101,2015-04-02T08:20:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,CARL,Carlton,Blues,CD_T30,BPL,CD_I294561,False,18.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
1586,CD_M20150140501,2015-05-01T09:50:00.000+0000,CONCLUDED,Premiership,Round 5,5,MCG,CARL,Carlton,Blues,CD_T30,BPR,CD_I294561,False,18.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
2289,CD_M20150140608,2015-05-10T05:20:00.000+0000,CONCLUDED,Premiership,Round 6,6,Marvel Stadium,CARL,Carlton,Blues,CD_T30,FB,CD_I294561,False,18.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
2508,CD_M20150140704,2015-05-16T06:35:00.000+0000,CONCLUDED,Premiership,Round 7,7,Marvel Stadium,CARL,Carlton,Blues,CD_T30,BPL,CD_I294561,False,18.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
7330,CD_M20150142006,2015-08-15T09:20:00.000+0000,CONCLUDED,Premiership,Round 20,20,Gabba,CARL,Carlton,Blues,CD_T30,BPR,CD_I294561,False,18.0,Kristian,Jaksch,FINAL_TEAM,away,NaN
7808,CD_M20150142108,2015-08-23T05:20:00.000+0000,CONCLUDED,Premiership,Round 21,21,MCG,CARL,Carlton,Blues,CD_T30,INT,CD_I294561,False,18.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
3914,CD_M20140141108,2014-06-01T06:40:00.000+0000,CONCLUDED,Premiership,Round 11,11,MCG,GWS,GWS Giants,Giants,CD_T1010,INT,CD_I294561,False,34.0,Kristian,Jaksch,FINAL_TEAM,away,NaN
4145,CD_M20140141205,2014-06-07T09:40:00.000+0000,CONCLUDED,Premiership,Round 12,12,GIANTS Stadium,GWS,GWS Giants,Giants,CD_T1010,HFFL,CD_I294561,False,34.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
5036,CD_M20140141407,2014-06-22T03:10:00.000+0000,CONCLUDED,Premiership,Round 14,14,GIANTS Stadium,GWS,GWS Giants,Giants,CD_T1010,INT,CD_I294561,False,34.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
5366,CD_M20140141505,2014-06-28T09:40:00.000+0000,CONCLUDED,Premiership,Round 15,15,SCG,GWS,GWS Giants,Giants,CD_T1010,INT,CD_I294561,False,34.0,Kristian,Jaksch,FINAL_TEAM,away,NaN


In [13]:
lineup_total

,providerId,utcStartTime,status,compSeason.shortName,round.name,round.roundNumber,venue.name,teamAbbr,teamName,teamNickname,teamId,position,player.playerId,player.captain,player.playerJumperNumber,player.playerName.givenName,player.playerName.surname,teamStatus,teamType,lateChanges
0,CD_M20150140101,2015-04-02T08:20:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,CARL,Carlton,Blues,CD_T30,BPL,CD_I294561,False,18.0,Kristian,Jaksch,FINAL_TEAM,home,NaN
1,CD_M20150140101,2015-04-02T08:20:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,CARL,Carlton,Blues,CD_T30,FB,CD_I261043,False,40.0,Michael,Jamison,FINAL_TEAM,home,NaN
2,CD_M20150140101,2015-04-02T08:20:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,CARL,Carlton,Blues,CD_T30,BPR,CD_I260278,False,17.0,Sam,Rowe,FINAL_TEAM,home,NaN
3,CD_M20150140101,2015-04-02T08:20:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,CARL,Carlton,Blues,CD_T30,HBFL,CD_I295518,False,15.0,Sam,Docherty,FINAL_TEAM,home,NaN
4,CD_M20150140101,2015-04-02T08:20:00.000+0000,CONCLUDED,Premiership,Round 1,1,MCG,CARL,Carlton,Blues,CD_T30,CHB,CD_I292511,False,42.0,Zach,Tuohy,FINAL_TEAM,home,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9517,CD_M20210142701,2021-09-25T09:15:00.000+0000,CONCLUDED,Premiership,Grand Final,27,Optus Stadium,WB,Western Bulldogs,Bulldogs,CD_T140,INT,CD_I992351,False,34.0,Bailey,Williams,FINAL_TEAM,away,NaN
9518,CD_M20210142701,2021-09-25T09:15:00.000+0000,CONCLUDED,Premiership,Grand Final,27,Optus Stadium,WB,Western Bulldogs,Bulldogs,CD_T140,INT,CD_I993834,False,5.0,Josh,Dunkley,FINAL_TEAM,away,NaN
9519,CD_M20210142701,2021-09-25T09:15:00.000+0000,CONCLUDED,Premiership,Grand Final,27,Optus Stadium,WB,Western Bulldogs,Bulldogs,CD_T140,INT,CD_I992054,False,37.0,Roarke,Smith,FINAL_TEAM,away,NaN
9520,CD_M20210142701,2021-09-25T09:15:00.000+0000,CONCLUDED,Premiership,Grand Final,27,Optus Stadium,WB,Western Bulldogs,Bulldogs,CD_T140,INT,CD_I993841,False,13.0,Josh,Schache,FINAL_TEAM,away,NaN


# Modelling

In [44]:
target_column = [
    'home_team_winner'
]

In [45]:
train_set = fixtures_usable_training[(fixtures_usable_training['year'] <= 2017) & (fixtures_usable_training['year'] != 2022)] # Only pas
test_set = fixtures_usable_training[fixtures_usable_training['year'] == 2022]

X_train = train_set.drop(columns=[target_column[0]])
X_test = test_set.drop(columns=[target_column[0]])

y_train = train_set[target_column]
y_test = test_set[target_column]

In [49]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

eval_set = [(X_test, y_test)]
clf = XGBClassifier(objective='binary:logistic', n_estimators=500000)

clf.fit(X_train, y_train, eval_metric="auc", eval_set=eval_set,
        early_stopping_rounds=2000,
        verbose=1000)

y_pred_train = clf.predict(X_train)

# evaluate predictions
train_accuracy = accuracy_score(y_train, y_pred_train)
print(f'Accuracy for classifier on train set: {train_accuracy}')

[0]	validation_0-auc:0.51770


/home/beefsports/mambaforge/envs/afl/lib/python3.11/site-packages/xgboost/sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/beefsports/mambaforge/envs/afl/lib/python3.11/site-packages/xgboost/sklearn.py:861: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1000]	validation_0-auc:0.52242
[2000]	validation_0-auc:0.52419
[3000]	validation_0-auc:0.52508
[4000]	validation_0-auc:0.52577
[5000]	validation_0-auc:0.52567
[6000]	validation_0-auc:0.52616
[6568]	validation_0-auc:0.52616
Accuracy for classifier on train set: 1.0


In [50]:
y_pred_test = clf.predict(X_test)

# evaluate predictions
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f'Accuracy for classifier on test set: {test_accuracy}')

Accuracy for classifier on test set: 0.529126213592233


In [51]:
y_pred_test_naive = y_test.replace(0, 1)
naive_accuracy = accuracy_score(y_test, y_pred_test_naive)
print(f'Accuracy for naive test set (home only): {naive_accuracy}')

Accuracy for naive test set (home only): 0.6019417475728155
